<a href="https://colab.research.google.com/github/fazal735/DL_A2/blob/main/DL_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# loading data from Kaggle to reduce the jargan of loading the data to gdrive and then loading or this ipynb
train_data_dir = '/kaggle/input/nature-12k/inaturalist_12K/train'
test_data_dir = '/kaggle/input/nature-12k/inaturalist_12K/val'


model definition

In [ ]:
# model definetion
class model(nn.Module):
  def __init__(self,num_filters=[32,64,128,256,512],filter_size=[3,3,5,5,7],activation=nn.ReLU(),
               stride=1, padding=1, pool_size=(2,2),fc_size=512,nom_o_classes=10,
               dropout=0,inchannels=3):
    super(model,self).__init__()
    self.num_filters=num_filters
    self.filter_size=filter_size
    self.activation=activation
    self.stride=stride
    self.padding=padding
    self.pool_size=pool_size
    self.fc_size=fc_size
    self.nom_o_classes=nom_o_classes
    self.dropout=dropout
    self.channels=inchannels


    def image_size(img_w,filter_size,padding,stride):
      return ((img_w-filter_size+2*padding)/stride+1)*0.5


    #layers of convolution
    #layer1
    self.conv_layer1=nn.Conv2d(self.channels,self.num_filters[0], stride=self.stride, padding=self.padding,
                               kernel_size=self.filter_size[0])
    self.dropout1=nn.Dropout2d(self.dropout)
    self.batch_norm1=nn.BatchNorm2d(self.num_filters[0])
    img_size1=image_size(224,self.filter_size[0],self.padding,self.stride)

    #layer2
    self.conv_layer2=nn.Conv2d(self.channels,self.num_filters[1], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[1])
    self.dropout2=nn.Dropout2d(self.dropout)
    self.batch_norm2=nn.BatchNorm2d(self.num_filters[1])
    img_size2=image_size(img_size1,self.filter_size[1],self.padding,self.stride)

    #layer3
    self.conv_layer3=nn.Conv2d(self.channels,self.num_filters[2], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[2])
    self.dropout3=nn.Dropout2d(self.dropout)
    self.batch_norm3=nn.BatchNorm2d(self.num_filters[2])
    img_size3=image_size(img_size2,self.filter_size[2],self.padding,self.stride)

    #layer4
    self.conv_layer4=nn.Conv2d(self.channels,self.num_filters[3], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[3])
    self.dropout4=nn.Dropout2d(self.dropout)
    self.batch_norm4=nn.BatchNorm2d(self.num_filters[3])
    img_size4=image_size(img_size3,self.filter_size[3],self.padding,self.stride)

    #layer5
    self.conv_layer5=nn.Conv2d(self.channels,self.num_filters[4], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[4])
    self.dropout5=nn.Dropout2d(self.dropout)
    self.batch_norm5=nn.BatchNorm2d(self.num_filters[4])
    img_size5=int(image_size(img_size4,self.filter_size[4],self.padding,self.stride))


    self.pool=nn.MaxPool2d(self.pool_size,stride=2)



    # forward
  def forward(self,x):
    #layer1
    x=self.conv_layer1(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout1(x)
    x=self.batch_norm1(x)

      #layer2
    x=self.conv_layer2(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout2(x)
    x=self.batch_norm2(x)

    #layer3
    x=self.conv_layer3(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout3(x)
    x=self.batch_norm3(x)

    #layer4
    x=self.conv_layer4(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout4(x)
    x=self.batch_norm4(x)

    #layer5
    x=self.conv_layer5(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout5(x)
    x=self.batch_norm5(x)


In [ ]:
epochs=100
learning_rate=0.1

In [ ]:
model=model()
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

model training function

In [ ]:
def training(model,data):
  loss_metric=nn.CrossEntropyLoss()
  optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

  model.train()
  training_loss=0.0
  true_label=0
  total_train=0

  for input, label in data:
    optimizer.zero_grad()
    output=model(input)
    loss=loss_metric(output,label)
    loss.backward()
    optimizer.step()
    trainning_loss+=loss.item()
    _,predicted=torch.max(output.data,1)
    total_train+=label.size(0)
    true_label+=(predicted==label).sum().item()

  train_accuracy=100*true_label/total_train
  return train_accuracy,training_loss.model


model testing function(on validation data)

In [ ]:
def test_on_valid_data(model, test_data):
    model.eval()
    correct_label = 0
    total_label = 0
    with torch.no_grad():
        for img, label in test_data:
            img, label = img.to(device), label.to(device)
            output = model(img)

            _, pred = torch.max(output, 1)
            correct_label += (pred == label).sum().item()
            total_label += label.size(0)

    valid_accuracy = 100 * correct_label / total_label
    return valid_accuracy


In [ ]:
def model_train_val(model, train_data, val_data,epochs):

    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model, avg_loss, train_accuracy = training(model, train_data)
        # Print training loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

        # Validation loop
        val_accuracy = test_on_valid_data(model, val_data)
        # Print validation accuracy
        print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')
